# 제곱 함수와 n 제곱 함수 만들기

In [1]:
# 제곱 함수
def my_sq(x):
    return x**2

# n 제곱 함수
def my_exp(x, n):
    return x**n

print(my_sq(4))
print(my_exp(2,4))

16
16


# 시리즈와 apply 메서드

#### apply 메서드 : 사용자가 작성한 함수를 한 번에 데이터프레임의 각 행과 열에 적용하여 실행할 수 있게 해주는 메서드

In [2]:
import pandas as pd

df = pd.DataFrame({'a':[10,20,30], 'b':[20,30,40]})
print(df)

    a   b
0  10  20
1  20  30
2  30  40


In [3]:
print(df['a']**2)

0    100
1    400
2    900
Name: a, dtype: int64


In [4]:
# 함수가 1개의 인자값일땐 인자값 생략
sq = df['a'].apply(my_sq)
print(sq)

0    100
1    400
2    900
Name: a, dtype: int64


In [5]:
# 2개의 인자값
ex = df['a'].apply(my_exp, n=2)
print(ex)

0    100
1    400
2    900
Name: a, dtype: int64


# 데이터 프레임과 apply 메서드

In [6]:
df = pd.DataFrame({'a':[10,20,30], 'b':[20,30,40]})
print(df)

    a   b
0  10  20
1  20  30
2  30  40


In [7]:
def print_me(x):
    print(x)

In [9]:
# axis : 함수 적용 방향 -> 열방향, 행방향
print(df.apply(print_me, axis=0))
print(df.apply(print_me, axis=1))

0    10
1    20
2    30
Name: a, dtype: int64
0    20
1    30
2    40
Name: b, dtype: int64
a    None
b    None
dtype: object
a    10
b    20
Name: 0, dtype: int64
a    20
b    30
Name: 1, dtype: int64
a    30
b    40
Name: 2, dtype: int64
0    None
1    None
2    None
dtype: object


In [10]:
# 이렇게 해서 열단위 데이터를 전달하면 1개만 인자로 받은거라 에러남
def avg_3(x,y,z): 
    return (x+y+z)/3

In [11]:
# 수정
def avg_3_apply(col):
    x = col[0]
    y = col[1]
    z = col[2]
    return (x+y+z)/3

print(df.apply(avg_3_apply, axis=0))

a    20.0
b    30.0
dtype: float64


In [12]:
# 데이터프레임의 행의 개수를 모를때 -> for문 사용
def avg_3_apply(col):
    sum = 0
    for item in col:
        sum+=item
    return sum/df.shape[0]

In [13]:
# 행 방향으로 데이터 처리하기
def avg_2_apply(row):
    sum = 0
    for item in row:
        sum+=item
    return sum/df.shape[1]

print(df.apply(avg_2_apply, axis=1))

0    15.0
1    25.0
2    35.0
dtype: float64


# 데이터프레임의 누락값을 처리한 다음 apply 메서드 사용하기 - 열 방향

In [14]:
# titanic 데이터
import seaborn as sns
titanic = sns.load_dataset("titanic")

In [15]:
print(titanic.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
survived       891 non-null int64
pclass         891 non-null int64
sex            891 non-null object
age            714 non-null float64
sibsp          891 non-null int64
parch          891 non-null int64
fare           891 non-null float64
embarked       889 non-null object
class          891 non-null category
who            891 non-null object
adult_male     891 non-null bool
deck           203 non-null category
embark_town    889 non-null object
alive          891 non-null object
alone          891 non-null bool
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB
None


In [16]:
# 누락값의 개수를 반환하는 함수 count_missing()

import numpy as np
def count_missing(vec):
    null_vec = pd.isnull(vec)
    null_count = np.sum(null_vec)
    return null_count

In [17]:
cmis_col = titanic.apply(count_missing)
print(cmis_col)

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64


In [18]:
# 누락값의 비율을 계산하는 함수 prop_missing()

def prop_missing(vec):
    num = count_missing(vec)
    dem = vec.size
    return num/dem

In [19]:
pmis_col = titanic.apply(prop_missing)
print(pmis_col)

survived       0.000000
pclass         0.000000
sex            0.000000
age            0.198653
sibsp          0.000000
parch          0.000000
fare           0.000000
embarked       0.002245
class          0.000000
who            0.000000
adult_male     0.000000
deck           0.772166
embark_town    0.002245
alive          0.000000
alone          0.000000
dtype: float64


In [20]:
# 누락값이 아닌 데이터 비율을 계산하는 함수 prop_complete()

def prop_complete(vec):
    return 1-prop_missing(vec)

# 데이터프레임의 누락값을 처리한 다음 apply 메서드 사용하기 - 행 방뱡

In [21]:
cmis_row = titanic.apply(count_missing, axis=1)
pmis_row = titanic.apply(prop_missing, axis=1)
pcom_row = titanic.apply(prop_complete, axis=1)

print(cmis_row.head())
print(pmis_row.head())
print(pcom_row.head())

0    1
1    0
2    1
3    0
4    1
dtype: int64
0    0.066667
1    0.000000
2    0.066667
3    0.000000
4    0.066667
dtype: float64
0    0.933333
1    1.000000
2    0.933333
3    1.000000
4    0.933333
dtype: float64


In [23]:
# 누락값의 개수를 구하여 새로운 열로 추가하기

titanic['num_missing'] = titanic.apply(count_missing, axis=1)
print(titanic.head())

   survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who  adult_male deck  embark_town alive  alone  num_missing  
0    man        True  NaN  Southampton    no  False            1  
1  woman       False    C    Cherbourg   yes  False            0  
2  woman       False  NaN  Southampton   yes   True            1  
3  woman       False    C  Southampton   yes  False            0  
4    man        True  NaN  Southampton    no   True            1  


In [24]:
# 누락값이 있는 데이터만 따로 모아서 보기
print(titanic.loc[titanic.num_missing>1,:].sample(10))

     survived  pclass     sex  age  sibsp  parch     fare embarked  class  \
593         0       3  female  NaN      0      2   7.7500        Q  Third   
335         0       3    male  NaN      0      0   7.8958        S  Third   
176         0       3    male  NaN      3      1  25.4667        S  Third   
653         1       3  female  NaN      0      0   7.8292        Q  Third   
667         0       3    male  NaN      0      0   7.7750        S  Third   
274         1       3  female  NaN      0      0   7.7500        Q  Third   
101         0       3    male  NaN      0      0   7.8958        S  Third   
524         0       3    male  NaN      0      0   7.2292        C  Third   
420         0       3    male  NaN      0      0   7.8958        C  Third   
180         0       3  female  NaN      8      2  69.5500        S  Third   

       who  adult_male deck  embark_town alive  alone  num_missing  
593  woman       False  NaN   Queenstown    no  False            2  
335    man    